In [27]:
class ElevatorStateMachine:
    STATES = {
        "отдых": {"отдых": lambda self: self, "подъем": lambda self: self.move_up, "спуск": lambda self: self.move_down},
        "открытие": {"отдых": lambda self: self.close_doors, "подъем": lambda self: self, "спуск": lambda self: self},
        "подъем": {"отдых": lambda self: self.stop_moving, "подъем": lambda self: self, "спуск": lambda self: self.stop_moving},
        "спуск": {"отдых": lambda self: self.stop_moving, "подъем": lambda self: self.stop_moving, "спуск": lambda self: self}
    }

    def __init__(self, num_floors, start_floor, elevator_id, history):
        self.num_floors = num_floors
        self.current_floor = start_floor
        self.commands = []
        self.id = elevator_id
        self.state = "отдых"
        self.history = history

    def process_floor_request(self, requested_floor):
        if requested_floor < 1 or requested_floor > self.num_floors:
            raise ValueError("Некорректный этаж")

        state_transition = self.STATES[self.state][self.get_transition_state(requested_floor)]
        state_transition(self)(requested_floor)

    def get_transition_state(self, requested_floor):
        if requested_floor > self.current_floor:
            return "подъем"
        elif requested_floor < self.current_floor:
            return "спуск"
        else:
            return "открытие"

    def open_doors(self):
        if self.state in ["подъем", "спуск"]:
            self.commands.append("Открыть дверь")
            self.history.append(f"{self.id + 1} лифт открыл двери на {self.current_floor} этаже.")
            self.state = "открытие"

    def close_doors(self):
        if self.state == "открытие":
            self.commands.append("Закрыть двери")
            self.history.append(f"{self.id + 1} лифт закрыл двери на {self.current_floor} этаже.")
            self.state = "отдых"

    def move_up(self, requested_floor):
        self.state = "подъем"
        self.commands.append("На этаж выше")
        self.history.append(f"{self.id + 1} лифт едет вверх: {self.current_floor}.")

        while self.current_floor < requested_floor:
            self.current_floor += 1
            self.history.append(f"{self.id + 1} лифт едет вверх: {self.current_floor}.")

        self.open_doors()

    def move_down(self, requested_floor):
        self.state = "спуск"
        self.commands.append("На этаж ниже")
        self.history.append(f"{self.id + 1} лифт едет вниз: {self.current_floor}.")

        while self.current_floor > requested_floor:
            self.current_floor -= 1
            self.history.append(f"{self.id + 1} лифт едет вниз: {self.current_floor}.")

        self.open_doors()

    def stop_moving(self, requested_floor):
        pass

    def reset(self):
        self.state = "отдых"
        self.commands = []

In [28]:
class ElevatorSystem:
    def __init__(self, num_floors, elevator_info):
        self.history = []
        self.elevators = [ElevatorStateMachine(num_floors, start_floor, i, self.history) for i, start_floor in enumerate(elevator_info)]

    def request(self, call_floor, target_floor):
        free_elevator = min(self.elevators, key=lambda elevator: len(elevator.commands))
        free_elevator.process_floor_request(call_floor)
        free_elevator.close_doors()
        free_elevator.process_floor_request(target_floor)
        free_elevator.close_doors()
        self.history.append(f"{free_elevator.id + 1} лифт совершил вызов по {call_floor} - {target_floor} этажам. \n")

    def run(self, requests):
        for elevator in self.elevators:
            elevator.reset()

        for request in requests:
            call_floor, target_floor = request
            self.request(call_floor, target_floor)

    def get_history(self):
        for step in self.history:
            print(step)

In [33]:
requests = [(1, 10), (3, 20), (5, 19), (19, 14), (8, 9), (17, 1)]
# ElevatorSystem(кол-во этвжей, начальные этажи кажого лифта)
elevator_system = ElevatorSystem(20, [2, 4, 1])
elevator_system.run(requests)
elevator_system.get_history()

1 лифт едет вниз: 2.
1 лифт едет вниз: 1.
1 лифт открыл двери на 1 этаже.
1 лифт закрыл двери на 1 этаже.
1 лифт едет вверх: 1.
1 лифт едет вверх: 2.
1 лифт едет вверх: 3.
1 лифт едет вверх: 4.
1 лифт едет вверх: 5.
1 лифт едет вверх: 6.
1 лифт едет вверх: 7.
1 лифт едет вверх: 8.
1 лифт едет вверх: 9.
1 лифт едет вверх: 10.
1 лифт открыл двери на 10 этаже.
1 лифт закрыл двери на 10 этаже.
1 лифт совершил вызов по 1 - 10 этажам. 

2 лифт едет вниз: 4.
2 лифт едет вниз: 3.
2 лифт открыл двери на 3 этаже.
2 лифт закрыл двери на 3 этаже.
2 лифт едет вверх: 3.
2 лифт едет вверх: 4.
2 лифт едет вверх: 5.
2 лифт едет вверх: 6.
2 лифт едет вверх: 7.
2 лифт едет вверх: 8.
2 лифт едет вверх: 9.
2 лифт едет вверх: 10.
2 лифт едет вверх: 11.
2 лифт едет вверх: 12.
2 лифт едет вверх: 13.
2 лифт едет вверх: 14.
2 лифт едет вверх: 15.
2 лифт едет вверх: 16.
2 лифт едет вверх: 17.
2 лифт едет вверх: 18.
2 лифт едет вверх: 19.
2 лифт едет вверх: 20.
2 лифт открыл двери на 20 этаже.
2 лифт закрыл двери